In [1]:
import numpy as np
import sframe as sf

In [2]:
products = sf.SFrame('../data/amazon_baby.gl/')

2016-03-14 21:43:06,930 [INFO] sframe.cython.cy_server, 172: SFrame v1.8.4 started. Logging /tmp/sframe_server_1458006186.log


In [3]:
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation)

In [4]:
products['review_clean'] = products['review'].apply(remove_punctuation)

In [5]:
products = products.fillna('review', '')

In [6]:
products = products[products['rating'] != 3]

In [7]:
products['sentiment'] = products['rating'].apply(lambda rating: +1 if rating > 3 else -1)

In [8]:
train_data, test_data = products.random_split(.8, seed = 1)

# Building the Bag Of Words aka Word Count aka Count Vector

Bag of words is the distinct collection of words in the corpus or training data. For each document or row, it is count of each word from the corpus.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')

# Convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review_clean'].to_numpy())
test_matrix = vectorizer.fit_transform(test_data['review_clean'].to_numpy())

# Train Sentiment Classifier With Logistic Regression

## Learn the Logistic Regression Classifier

In [11]:
from sklearn.linear_model import LogisticRegression
sentiment_model = LogisticRegression()

sentiment_model.fit(X = train_matrix, y = train_data['sentiment'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [12]:
coeff = sentiment_model.coef_
pos_wts = coeff[ coeff >= 0 ]
print "Ans: Weights >= 0:", np.shape(pos_wts)[0]

Ans: Weights >= 0: 86906


### Making Predictions With Logistic Regression

In [13]:
sample_test_data = test_data[10:13]
print sample_test_data

+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|   Our Baby Girl Memory Book   | Absolutely love it and all... |  5.0   |
| Wall Decor Removable Decal... | Would not purchase again o... |  2.0   |
| New Style Trailing Cherry ... | Was so excited to get this... |  1.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+-----------+
|          review_clean         | sentiment |
+-------------------------------+-----------+
| Absolutely love it and all... |     1     |
| Would not purchase again o... |     -1    |
| Was so excited to get this... |     -1    |
+-------------------------------+-----------+
[3 rows x 5 columns]



In [14]:
sample_test_data[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

In [15]:
sample_test_data[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

In [16]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'].to_numpy())

In [17]:
scores = sentiment_model.decision_function(sample_test_matrix)
print scores

ValueError: X has 49781 features per sample; expecting 121712